# This is an example notebook on how to use NAM for shape matching
In this notebook we consider different refinement with the spectral embeddings


In [2]:
import os
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
from geomfum.shape.mesh import TriangleMesh
from geomfum.refine import ZoomOut

import torch
import numpy as np

import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from model.neural_adjoint_map import NeuralAdjointMap
from sklearn.neighbors import NearestNeighbors

from model.neural_zoomout import NeuralZoomOut


In [3]:
import os
from urllib.request import urlretrieve

faust_url = "https://raw.githubusercontent.com/JM-data/PyFuncMap/4bde4484c3e93bff925a6a82da29fa79d6862f4b/FAUST_shapes_off/"
shape_files = ["tr_reg_080.off", "tr_reg_093.off"]
for fname in shape_files:
    url = faust_url + fname
    out_path = os.path.join("../data/", fname)
    urlretrieve(url, out_path)


In [4]:
mesh1 = TriangleMesh.from_file("../data/tr_reg_080.off")
mesh2 = TriangleMesh.from_file("../data/tr_reg_093.off")

eigvals1, eigvecs1 = mesh1.laplacian.find_spectrum(spectrum_size=200)
eigvals2, eigvecs2 = mesh2.laplacian.find_spectrum(spectrum_size=200)

c:\Users\giuli\OneDrive\Research\NAM-NeuralAdjointMaps\venv\Lib\site-packages\geomfum\_backend\pytorch\sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


In [5]:
p2p_gt = np.arange(mesh1.n_vertices)

# Initial Map

In [7]:
from geomfum.convert import FmFromP2pConverter, P2pFromFmConverter


fmap_from_p2p=FmFromP2pConverter()
p2p_from_fmap = P2pFromFmConverter()

mesh1.basis.use_k=20
mesh2.basis.use_k=20
fmap = fmap_from_p2p(p2p_gt, mesh1.basis, mesh2.basis)


p2p_ini = p2p_from_fmap(fmap, mesh1.basis, mesh2.basis)


print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p_ini] - mesh2.vertices[p2p_gt]) ** 2),
)

Conversion Error: tensor(0.0011)


# ZoomOut

In [8]:
mesh1.basis.use_k=200
mesh2.basis.use_k=200
zoomout = ZoomOut(nit=9, step=20)
ref_fmap = zoomout(fmap, mesh1.basis, mesh2.basis)

In [9]:
p2p = p2p_from_fmap(ref_fmap, mesh1.basis, mesh2.basis)

print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p] - mesh2.vertices[p2p_gt]) ** 2),
)

Conversion Error: tensor(0.0002)


# NeuralZoomOut

In [10]:
from model.neural_zoomout import NamFromP2pConverter, P2pFromNamConverter, NeuralZoomOut

nam_from_p2p=NamFromP2pConverter()
p2p_from_nam = P2pFromNamConverter()

mesh1.basis.use_k=20
mesh2.basis.use_k=20
nam=nam_from_p2p(p2p_gt, mesh1.basis, mesh2.basis)


mesh1.basis.use_k=200   
mesh2.basis.use_k=200
nzo = NeuralZoomOut(nit=9, step=20)
ref_nam = nzo(nam, mesh1.basis, mesh2.basis)
p2p = p2p_from_nam(ref_nam, mesh1.basis, mesh2.basis)
print(
    "Conversion Error:",
    gs.mean((mesh2.vertices[p2p] - mesh2.vertices[p2p_gt]) ** 2),
)

Conversion Error: tensor(0.0002)
